In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-csv_2.11:1.4.0,org.apache.spark:spark-streaming-kafka_2.11:1.6.1,com.datastax.spark:spark-cassandra-connector_2.11:1.6.0-M2 pyspark-shell'
import time

In [108]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row
#sc.stop()
conf = SparkConf() \
    .setAppName("Kafka to Cassandra") \
    .setMaster("local[4]") \
    .set("spark.cassandra.connection.host", "localhost")

sc = SparkContext(conf=conf)

In [109]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [130]:
ssc.stop()
sc.stop()
sc = SparkContext(conf=conf)

sqlContext=SQLContext(sc)
ssc = StreamingContext(sc, 5)

In [131]:
def processRawTransaction(transaction):
    (card_pan_id,term_miduid,term_mid,term_uid,term_mcc,term_country,tx_amount,\
    tx_datetime,tx_accepted,tx_ecom,tx_3d_secure,tx_emv,age,language,\
     gender,broker,card_brand,tx_fraud)\
    =transaction.split(',')
    
    transactionRow=Row(\
        card_pan_id=card_pan_id,\
        term_miduid=term_miduid,\
        term_mid=term_mid,\
        term_uid=term_uid,\
        term_mcc=term_mcc,\
        term_country=term_country,\
        tx_amount=tx_amount,\
        tx_datetime=tx_datetime,\
        tx_accepted=tx_accepted,\
        tx_ecom=tx_ecom,\
        tx_3d_secure=tx_3d_secure,\
        tx_emv=tx_emv,\
        age=age,\
        language=language,\
        gender=gender,\
        broker=broker,\
        card_brand=card_brand,\
        tx_fraud=tx_fraud\
       )
    return transactionRow
    

In [135]:
def saveToCassandra(rows):
    if not rows.isEmpty(): 
        sqlContext.createDataFrame(rows).write\
        .format("org.apache.spark.sql.cassandra")\
        .mode('append')\
        .options(table="rawdata", keyspace="tx")\
        .save()


In [133]:
kvs = KafkaUtils.createStream(ssc, "127.0.0.1:2181", "spark-streaming-consumer", {'test': 1})

#Get raw data from transaction stream
rawTransactionData = kvs.map(lambda x: x[1])

#Convert raw data into typed Row objects
transactionData=rawTransactionData.map(processRawTransaction)

#Save rows to Cassandra
transactionData.foreachRDD(saveToCassandra)

ssc.start()

In [129]:
ssc.stop()